In [1]:
import pandas as pd
import numpy as np
import os, re, glob, pyproj, math, datetime
from sys import platform
import datatable as dt
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import re

In [ ]:
# read in data
if platform == "darwin":
    fileloc = "/Volumes/GoogleDrive/My Drive/PhD/Data/2018Shearwater/TxtDat/AxyTrek/AlgorithmOutputUpdate/PredictedForage/"
else:
    fileloc = "E:/My Drive/PhD/Data/2018Shearwater/TxtDat/AxyTrek/AlgorithmOutputUpdate/PredictedForage/"

files = glob.glob(fileloc+"*ForageGPS.txt")

forDat = pd.DataFrame()
for b in files:
    new = pd.read_csv(b)
    new.insert(0,'ID',re.findall('[0-9\-]+',re.search('.*PredictedForage(.+?)_S*',b).group(1)) * len(new))
    forDat = forDat.append(new,ignore_index=True)

forDat.loc[:,'DT'] = pd.to_datetime(forDat.loc[:,'DT'],format="%d/%m/%Y %H:%M:%S.%f")
# wind data
if platform == "darwin":
    windloc = "/Volumes/GoogleDrive/My Drive/PhD/Data/2018Shearwater/WindEst/MinDat/"
else:
    windloc = "E:/My Drive/PhD/Data/2018Shearwater/WindEst/MinDat/"

wfiles = glob.glob(windloc+"*.csv")
wDat = pd.DataFrame()
for b in wfiles:
    new = pd.read_csv(b,names=["DT","Lat","Lon","BHead","X","Y"])
    new.insert(0,"ID",re.findall('[0-9\-]+',re.search('.*MinDat(.+?)_S*',b).group(1)) * len(new))
    wDat = wDat.append(new,ignore_index=True)
wDat.loc[:,'DT'] = pd.to_datetime(forDat.loc[:,'DT'],format="%d/%m/%Y %H:%M:%S")

: 

In [3]:
# calculate the distance & time from each wind calculation to the next foraging point
def DistTimeTo(dt,lat,lon,id,dat):
    try:
        forPoint = np.nanmin(np.where((dat['Forage'] == 1) & (dat['ID'] == id) & (dat['DT'] > dt)))
        tdiff = (dat.loc[forPoint,'DT'] - dt)/np.timedelta64(1,'s')
        azimuth1, azimuth2, distance = pyproj.Geod(ellps='WGS84').inv(dat.loc[forPoint,'Lon'],dat.loc[forPoint,'Lat'],lon,lat)
    except:
        tdiff = float("NaN")
        distance=float("NaN")
    return tdiff,distance

In [4]:
tst = [DistTimeTo(wDat.loc[b,'DT'],wDat.loc[b,'Lat'],wDat.loc[b,'Lon'],wDat.loc[b,'ID'],forDat) for b in range(len(wDat))]

In [6]:
tst

[(374.0, 434079.43633166136),
 (359.0, 433741.2928513437),
 (329.0, 433358.4708119085),
 (299.0, 432962.8070911132),
 (267.0, 432600.9748653279),
 (239.0, 432183.3213832057),
 (209.0, 414517.6352703229),
 (179.0, 414072.3012452631),
 (148.0, 413674.60272704996),
 (119.0, 413278.77286610316),
 (88.0, 412843.8956371407),
 (59.0, 412484.6533407217),
 (29.0, 412095.11022444523),
 (27.0, 411704.4941690057),
 (386.0, 411831.260338359),
 (359.0, 411454.12652994413),
 (327.0, 411033.33110202226),
 (299.0, 410602.9926734411),
 (269.0, 410227.1499922777),
 (239.0, 409793.03807194886),
 (208.0, 409408.32348469144),
 (178.0, 408991.6921407225),
 (148.0, 408544.29719101207),
 (118.0, 408109.54743811395),
 (89.0, 407681.05083371216),
 (57.0, 407309.49751523964),
 (27.0, 406933.4042972179),
 (28.0, 406517.0013102986),
 (29.0, 406115.58325342176),
 (28.0, 405729.19317729585),
 (90.0, 405395.2092777441),
 (56.0, 405006.8388619482),
 (26.0, 404587.56946488644),
 (30.0, 404156.9648979269),
 (989.0, 40375

In [30]:
wDat.loc[:,6]

KeyError: 6

In [122]:
testPhrase=wfiles[0][0:wfiles[0].find("_S")-1]+"(.?)_S"
# re.search(testPhrase,wfiles[0]).group(1)
testPhrase

'E:/My Drive/PhD/Data/2018Shearwater/WindEst/MinDat\\(.?)_S'